In [ ]:
# install neccesary plugins
!pip install pandas
!pip install selenium
!pip install webdriver_manager
!pip install bs4

In [ ]:
# imports
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from datetime import datetime
from webdriver_manager.firefox import GeckoDriverManager

# update and run the latest Firefox webdriver
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

In [ ]:
# dicationary to turn into final table
defi_dict = {}

# class ids
protocol = 'sc-caSCKo hKzJw'
token = 'sc-caSCKo dIpawx'
platform = 'sc-ksYbfQ jqDzNU css-4cffwv'
tvl = 'sc-ksYbfQ jqDzNU css-4cffwv'
html_id = 'sc-cHGsZl dBFEZZ css-1yh09yi'

# navigation xpaths
xpath_wait = "/html/body/div/div/div/div[2]/div/div/div[2]/div/div[1]/div[1]/div"
xpath_next_page = '/html/body/div/div/div/div[2]/div/div/div[2]/div/div[4]/div[3]/div'
xpath_pagenumber = '/html/body/div/div/div/div[2]/div/div/div[2]/div/div[4]/div[2]'

# sites to iterate through
site_dict = {'Protocols':'https://defillama.com/protocols',
             'Dexes':'https://defillama.com/protocols/dexes',
             'Assets':'https://defillama.com/protocols/assets',
             'Lending':'https://defillama.com/protocols/lending',
             'Yield':'https://defillama.com/protocols/yield',
             'Insurance':'https://defillama.com/protocols/insurance',
             'Options':'https://defillama.com/protocols/options',
             'Indexes':'https://defillama.com/protocols/indexes',
             'Staking':'https://defillama.com/protocols/staking'}


for item in site_dict.items():
    driver.get(item[1])
    
    # wait until the 'Name' element of the site has been loaded
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_wait)))
    except:
        pass
    
    number_of_pages = int(driver.find_element_by_xpath(xpath_pagenumber).text.split()[-1:][0])
 
    # iterate through the pages
    for page in range(1,number_of_pages +1):

        # grab the html of each table row
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        html = soup.find_all("div", class_ = html_id)
        rows = html[0].find_all('div', attrs ={'style': 'height: 48px;'})

        # grab the elements from the different columns in the row
        for rownumber in range(len(rows)):
            defi_dict[rows[rownumber].find_all('div', attrs={'class': protocol})[0].text] = \
                 [item[0],
                  rows[rownumber].find_all('div', attrs={'class': token})[0].text, 
                  rows[rownumber].find_all('div', attrs={'class': platform})[0].text,
                  rows[rownumber].find_all('div')[11].text,
                  rows[rownumber].find_all('div')[13].text,
                  rows[rownumber].find_all('div')[9].text,
                  rows[rownumber].find_all('div', attrs={'class': tvl})[2].text]
        
        # go to the next page
        driver.find_element_by_xpath(xpath_next_page).click()


In [ ]:
# convert dictionary into Pandas dataframe
df = pd.DataFrame.from_dict(defi_dict, 
                            orient='index',
                            columns = ['Category',
                                       'Symbol',
                                       'Chain',
                                       '1h Change',
                                       '1d Change',
                                       '7d Change',
                                       'TVL'])

# partially clean TVL text
df['TVL_converted'] = df['TVL'].str.replace('$','', regex = True).replace(',','', regex = True)

df['Snapshot_taken'] = datetime.now()

df = df.reset_index().rename(columns = {'index':'Protocol'})
df

In [ ]:
# further clearn the TVL text
def value_to_float(x):
    if 'b' in x:
        return round(float(x.replace('b', '')) * 1000000000,0)
    if 'm' in x:
        return round(float(x.replace('m', '')) * 1000000,0)
    else:
        return int(x)

df['TVL_converted'] = df['TVL_converted'].apply(value_to_float)
df['TVL_converted'] = df['TVL_converted'].astype('Int64')

In [ ]:
df.to_csv('Defi llama stats ' + datetime.now().strftime('%Y.%m.%d - %Hh%Mmin') + '.csv',
          index = False)